In [2]:
using Plots
using TensorCrossInterpolation, LinearAlgebra
using QuanticsTCI
using QuanticsGrids
using ITensorMPS
using ITensors
using NDTensors
using TCIITensorConversion
using IterativeSolvers
using LinearMaps
using LinearAlgebra
include("src/differential_mpo.jl")

get_c_vec (generic function with 1 method)

In [3]:
# simulation parameter

R = 10
max_bond = 10
delta_t = 0.1 * 2.0^-(R-1)
v = 1e-5
penalty_coefficient = 2.5e5


# starting conditions and boundaries

a = 0.1
xmin = 0
xmax = 1
ymin = 0
ymax = 1
dx = float(abs(xmin - xmax) / (2^R-1))
dy = float(abs(ymin - ymax) / (2^R-1))

function u_1(x, y)
    return exp(-((((x - 0.5)/a)^2/2) + (((y - 0.5)/a)^2)/2))
end

function u_2(x, y)
    return exp(-((((x - 0.5)/a)^2/2) + (((y - 0.5)/a)^2)/2))
end

# QuanticsTCI
qgrid = DiscretizedGrid{2}(R, (xmin, ymin), (xmax, ymax); includeendpoint = true)
u1q, ranks1, errors1 = quanticscrossinterpolate(Float64, u_1, qgrid; maxbonddim=max_bond)
u2q, ranks2, errors2 = quanticscrossinterpolate(Float64, u_2, qgrid; maxbonddim=max_bond)

(QuanticsTCI.QuanticsTensorCI2{Float64}(TensorCrossInterpolation.TensorCI2{Float64} with rank 10, DiscretizedGrid{2}(10, (0.0, 0.0), (1.0, 1.0), 2, :fused, true), TensorCrossInterpolation.CachedFunction{Float64, BitIntegers.UInt256} with 16965 entries), [15, 15, 15], [0.041509875322360175, 0.03634987042325569, 0.03634987042325569])

In [4]:
# Convert to MPS Format
s = siteinds("Qudit", R, dim=4)
u = [ITensorMPS.MPS(TensorTrain(u1q.tci), sites=s), ITensorMPS.MPS(TensorTrain(u2q.tci), sites=s)]
u_p = deepcopy.(u)

# Build Diff MPOs
d1 = [Diff_1_8_x(dx, s), Diff_1_8_y(dy, s)]
d2 = [Diff_2_8_x(dx, s), Diff_2_8_y(dy, s)]
del = MPO([delta(s[i], s[i]', s[i]'') for i in 1:length(s)])
print("Done!")

Done!

In [5]:
center = 2
u = orthogonalize.(u, center)
u_p = orthogonalize.(u_p, center)
c_vec = get_c_vec(u_p, center)
beta = make_beta(u, u_p, center, delta_t, v, d1, d2, del, max_bond)

function A_function(c_vec)
    return c_vec - penalty_coefficient * delta_t^2 * apply_H(c_vec, u_p, d1, center, max_bond)
end
d = sum(length.(array.(getindex.(u_p, center))))
A = FunctionMap{Float64,false}(A_function, d)

320×320 FunctionMap{Float64,false}(A_function; issymmetric=false, ishermitian=false, isposdef=false)

In [6]:
A_dense = Matrix(A)

320×320 Matrix{Float64}:
 96.8511       -4.21775      -2.40202e-14  …   -0.303938       0.00678423
 -4.20996      96.8511       -1.01173e-14       0.0310411     -0.303938
 -5.59524e-16   4.34712e-15  96.8511           -9.44955e-17   -1.46344e-16
 -4.20877e-15   9.88142e-15  -4.20996          -8.39314e-17   -4.82901e-17
  0.0           0.0           0.0               8.50282e-18    0.0120663
  0.0           0.0           0.0          …    0.00190936    -2.80646e-17
  0.0           0.0           0.0              -5.08164e-16    0.0832631
  0.0           0.0           0.0              -0.00468119     1.7002e-16
 -3.85592e-15   4.20996      -8.50288e-17      -1.59534e-17   -0.0310411
  4.21775      -1.38877e-14   6.9677e-17       -0.00678423    -9.60116e-17
  ⋮                                        ⋱                 
 -0.0819891     1.64972e-17  -7.16305e-18       1.8179e-31     1.14629e-18
 -5.65298e-17  -0.0163453     1.30905e-16      -4.63951e-15    5.65407e-35
 -0.0877107    -2.93948e

In [7]:
eigs = eigvals(A_dense)
display(eigs)

320-element Vector{ComplexF64}:
 0.8767756987751303 - 0.06495156566449842im
 0.8767756987751303 + 0.06495156566449842im
 0.8770145596235822 - 0.011883663315392888im
 0.8770145596235822 + 0.011883663315392888im
 0.9333773956724343 - 0.045413296900522857im
 0.9333773956724343 + 0.045413296900522857im
 0.9433659041339276 - 0.03988096034005684im
 0.9433659041339276 + 0.03988096034005684im
 0.9493583179956113 - 0.06568449427150634im
 0.9493583179956113 + 0.06568449427150634im
                    ⋮
 1246.0667295648213 + 0.3604886495201543im
 1246.0675363268201 - 0.3641245723068253im
 1246.0675363268201 + 0.3641245723068253im
 1246.0714206048797 - 0.36345828176836936im
 1246.0714206048797 + 0.36345828176836936im
 1374.8290908982194 + 0.0im
 1374.8322239638073 - 0.0002237238831146868im
 1374.8322239638073 + 0.0002237238831146868im
  1374.835075750769 + 0.0im

In [8]:
println(c_vec)
# solve system
history = cg!(c_vec, A, beta, abstol=0, reltol=1e-4, verbose=false, maxiter=100, log=true)[2]
println("Center $center: $history")
println(c_vec)
norm(A(c_vec) - beta)

[-90.81929910588754, -1.47461335470254, -1.5608826081070604, 0.036196833086369146, -0.0017495566935689284, -0.42242588991883256, -0.046902463233209464, -12.51458168108711, 0.14326298273485727, 34.62851996660882, 0.006094169432335344, 1.3711161560847964, 0.15543231474843058, -0.031666887251282784, 34.64652484594009, 0.48188093546699134, 0.0, 0.16117218686686177, 0.181507759238379, 0.012366731972950054, -0.06226545405727983, -3.782437032939482, -1.4634441333032062, -89.72348063729241, 0.5166151674430631, 30.328449590474193, 0.019538190339922414, 1.1341245542456824, 0.4718887043005652, -0.029801711863413374, 30.22455282039983, 0.11596604980110069, 0.0, -0.40910534742280175, -0.0034760629283282427, -0.018541428918616393, -0.004317128798523434, -0.13371130227489286, -0.03429560285645105, 0.0, -1.487218794929065, -78.24370613120456, -0.06434682119137765, -3.3807639313828974, 0.1994409505470569, -0.006266193218424021, 12.518744911790222, 0.2053358584230447, 0.0, -0.09665038646427754, -0.45091

477.9501465322459